In [ ]:
import numpy as np
import pandas as pd
import datetime

In [ ]:
df = pd.read_csv('../input/flight-delays/flights.csv')

In [ ]:
df.head()

In [ ]:
df['DATE'] = pd.to_datetime(df[['YEAR','MONTH', 'DAY']])

In [ ]:
df.drop(['CANCELLATION_REASON','AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY','TAIL_NUMBER','CANCELLED','DAY_OF_WEEK','YEAR','MONTH','DAY',
        'TAXI_OUT','WHEELS_OFF','AIR_TIME','WHEELS_ON','TAXI_IN','ELAPSED_TIME','SCHEDULED_TIME'],
        axis=1,inplace=True)

In [ ]:
df = df.loc[(df['AIRLINE']=='AS') | (df.AIRLINE=='NK') | (df.AIRLINE=='VX') | (df.AIRLINE=='HA') | (df.AIRLINE=='F9')]
df.dropna(inplace=True)

In [ ]:
def format_time(item):
    if pd.isnull(item):
        return np.nan
    else:
        if item == 2400: item=0
        item = "{0:04d}".format(int(item))
        time = datetime.time(int(item[:2]),int(item[2:4]))
        return time

df['DEPARTURE_TIME'] = df['DEPARTURE_TIME'].apply(format_time)
df['SCHEDULED_ARRIVAL'] = df['SCHEDULED_ARRIVAL'].apply(format_time)
df['ARRIVAL_TIME'] = df['ARRIVAL_TIME'].apply(format_time)
df['SCHEDULED_DEPARTURE'] = df['SCHEDULED_DEPARTURE'].apply(format_time)

In [ ]:
df['Initial_Fuel'] = 0
df['Initial_Fuel'] = df['Initial_Fuel'].apply(lambda item: np.random.randint(2320,2450))

In [ ]:
df.head()

In [ ]:
a = pd.DataFrame(df[(df.ORIGIN_AIRPORT=='ANC') & (df.AIRLINE=='AS') & (df.DATE=='2015-01-01')])
a['REMAINING_FUEL'] = 0.0
a.head()

In [ ]:
#Normal Queue Optimization
airlines = {}
for i in range(5):
    key=a.ORIGIN_AIRPORT.iloc[i]
    if key not in airlines:
        airlines[key] = [[a.FLIGHT_NUMBER.iloc[i],a.DESTINATION_AIRPORT.iloc[i],a.DISTANCE.iloc[i],a.Initial_Fuel.iloc[i]]]
    else:
        airlines[key].append([a.FLIGHT_NUMBER.iloc[i],a.DESTINATION_AIRPORT.iloc[i],a.DISTANCE.iloc[i],a.Initial_Fuel.iloc[i]])
airlines

In [ ]:
#Fuel Calculation
flag = 5 #Queue Waiting Time
for item in range(len(airlines['ANC'])):
    row = airlines['ANC'][item]
    a['REMAINING_FUEL'][item] = row[3] - (row[2] * 1.5) - flag
    flag += 5

In [ ]:
'''airlines = {}
def queue(origin_airport, airline, date):
    temp = pd.DataFrame(df[(df.ORIGIN_AIRPORT==origin_airport) & (df.AIRLINE==airline) & (df.DATE==date)])
    for i in range(len(temp)):
        key = temp.ORIGIN_AIRPORT.iloc[i]
        if key not in airlines:
            airlines[key] = [[temp.FLIGHT_NUMBER.iloc[i],temp.DESTINATION_AIRPORT.iloc[i],temp.DISTANCE.iloc[i],temp.Initial_Fuel.iloc[i]]]
        else:
            airlines[key].append([temp.FLIGHT_NUMBER.iloc[i],temp.DESTINATION_AIRPORT.iloc[i],temp.DISTANCE.iloc[i],temp.Initial_Fuel.iloc[i]])
        
queue('ANC','AS','2015-01-01')
airlines'''

In [ ]:
'''flag = 3 #Queue Waiting Time
for item in range(len(airlines['ANC'])):
    row = airlines['ANC'][item]
    a['REMAINING_FUEL'][item] = row[3] - (row[2] * 1.5) - flag
    flag += 3'''

In [ ]:
a['DIVERTED'] = a['DIVERTED'].apply(lambda item: np.random.randint(0,2))

In [ ]:
#Emergency Queue Optimization
emergency_queue={}
normal_queue = {}
for i in range(len(a)):
    key=a.ORIGIN_AIRPORT.iloc[i]
    check=a.DIVERTED.iloc[i]
    if(check==1):
        if key not in emergency_queue:
            emergency_queue[key] = [[a.FLIGHT_NUMBER.iloc[i],a.DESTINATION_AIRPORT.iloc[i],a.SCHEDULED_DEPARTURE.iloc[i].strftime('%H:%M:%S'),a.Initial_Fuel.iloc[i]]]
          
        else:
            emergency_queue[key].append([a.FLIGHT_NUMBER.iloc[i],a.DESTINATION_AIRPORT.iloc[i],a.SCHEDULED_DEPARTURE.iloc[i].strftime('%H:%M:%S'),a.Initial_Fuel.iloc[i]])
            
    else:
        if key not in normal_queue:
            normal_queue[key] = [[a.FLIGHT_NUMBER.iloc[i],a.DESTINATION_AIRPORT.iloc[i],a.SCHEDULED_DEPARTURE.iloc[i].strftime('%H:%M:%S'),a.Initial_Fuel.iloc[i]]]
        else:
            normal_queue[key].append([a.FLIGHT_NUMBER.iloc[i],a.DESTINATION_AIRPORT.iloc[i],a.SCHEDULED_DEPARTURE.iloc[i].strftime('%H:%M:%S'),a.Initial_Fuel.iloc[i]])
            

print(emergency_queue)
print("-----------")
print(normal_queue)

In [ ]:
df1 = pd.read_csv('../input/flight-delays/airports.csv')
df1.head()

In [ ]:
d = {}
for item in range(len(df1)):
    desti = df1.IATA_CODE.iloc[item]
    if desti not in d:
        d[desti] = [df1.LATITUDE.iloc[item],df1.LONGITUDE.iloc[item]]

In [ ]:
'''df['LATITUDE'] = 0.0
df['LONGITUDE'] = 0.0
for item in range(len(df)):
    desti = df.DESTINATION_AIRPORT.iloc[item]
    if desti in d:
        df['LATITUDE'][item] = d[desti][0]
        df['LONGITUDE'][item] = d[desti][1]'''

In [ ]:
a.reset_index(inplace = True, drop = True)
a.to_csv('Emergency_Data.csv',index=False)